In [2]:
import pandas as pd


In [4]:
def load_datasets():
    nasdaq_data = pd.read_csv("datasets/NASDAQ_100.csv")
    QQQ_data = pd.read_csv("datasets/QQQ_raw.csv")
    QQQ_split_data = pd.read_csv("datasets/QQQ_split_adj.csv")
    SPY500_data = pd.read_csv("datasets/SP500.csv")
    SPY_data = pd.read_csv("datasets/SPY.csv")

    # Printing the column names for each dataset
    print("NASDAQ 100 Columns:", nasdaq_data.columns.tolist())
    print("QQQ Raw Columns:", QQQ_data.columns.tolist())
    print("QQQ Split Adjusted Columns:", QQQ_split_data.columns.tolist())
    print("S&P 500 Columns:", SPY500_data.columns.tolist())
    print("SPY Columns:", SPY_data.columns.tolist())

load_datasets()
    

NASDAQ 100 Columns: ['date', 'open', 'high', 'low', 'close', 'volume', 'change_percent', 'avg_vol_20d']
QQQ Raw Columns: ['date', 'open', 'high', 'low', 'close', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']
QQQ Split Adjusted Columns: ['date', 'open', 'high', 'low', 'close', 'volume', 'raw_close', 'change_percent', 'avg_vol_20d']
S&P 500 Columns: ['date', 'open', 'high', 'low', 'close', 'volume', 'change_percent', 'avg_vol_20d']
SPY Columns: ['date', 'open', 'high', 'low', 'close', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']
